# Задача: реализовать пакетный градиентный спуск с ранним прекращением для полиномиальной логистической регрессии (не используя Scikit-Learn).

## Для начала разберемся с терминами  

*Полиномиальная логистическая регрессия*  

Для начала, разберемся, что такое логистическая регрессия  

Логистическая регрессия или логит-регрессия - алгоритм регрессии, который может применяться для задач классификации. Он используется для оценки вероятности принадлежности экземпляра к определенному классу. Логистическая регрессионная модель подсчитывает взвешенные суммы входных признаков и выдает логистику результата.

Векторизированная форма для оценки вероятности принадлежности объекта к данному классу
\begin{align}
\hat{p}=h_\theta(x)=\sigma(\theta^T\cdotp x)
\end{align}

Это сигмоидальная (S-образная) функция, которая выдает значения на интервале от 0 до 1 и определяется выражением.
\begin{align}
\sigma(t)=\frac{1}{1+exp(-t)}
\end{align}

прогноз логистической регрессионной модели:
\begin{align}
\hat{y}=\begin{cases} 0,\; if\; \hat{p} < 0.5,\\
1,\; if\; \hat{p} \geq 0.5. \end{cases}
\end{align}

Функция издержек одиночного обучающего образца
\begin{align}
c(\theta)=\begin{cases} -log(\hat{p}),\; if\;y=1,\\
-log(1-\hat{p}),\; if\; y=0. \end{cases}
\end{align}

А когда логит-регрессия становится полиномиальной?  

Логистическая регрессионная модель может быть обобщена для поддержки множества классов напрямую, без необходимости в обучении и комбинировании многочисленных двоичных классификаторов. В результате получается полиномиальная или многопеременная логистическая регрессия.
Имея образец $x$, полиномиальная логистическая регрессионная модель сначала вычисляет сумму очков $s_k(x)$ для каждого класса $k$, а затем оценивает вероятность каждого класса, применяя к суммам очков полиномиальную логистическую функцию (softmax), также называемую нормализованной экспоненциальной (normalized exponential).  
Полиномиальная логистическая сумма очков для класса $k$:
\begin{align}
s_k(x)=(\theta^{(k)})^T\cdotp x
\end{align}

После получения вектора для каждого класса образца $x$ оценивается вироятность принадлежности к классу $k$ - она вычиляет экспоненту каждой суммы очков и затем нормализует их:
\begin{align}
\hat{p}_k=\sigma(s(x))_k=\frac{exp(s_k(x))}{\sum_{j=1}^kexp(s_j(x))}
\end{align}
где:  
$K$ - количество классов;  
$s(x)$ - вектор, содержащий суммы очков каждого класса для образца x;
$\sigma(s(x))_k$ - оценочная вероятность того, что образец $x$ принадлежит классу $k$ при заданных суммах очков каждого класса этог образца.

Подобно классификатору, основанному на логистической регрессии, классификатор на базе многопеременной логистической регрессии прогнозирует класс с наивысшей оценочной вероятностью (который является просто классом с самой высокой суммой очков:
\begin{align}
\hat{y} = \arg\max_k\sigma(s(x))_k=\arg\max_ks_k(x)=\arg\max_k((\theta^{(k)})^T\cdotp x)
\end{align}

Операция argmax возвращает значение переменной, которая обращает функцию в максимум.  
Классификатор на полиномальной логистической регрессии прогнозирует только один класс за раз, поэтому должен использоваться только с взаимоисключающими классами.  


В качестве функции издержек используется функция перекрестной энтропии (кросс-энотропии), которая страфует модель, когда та дает оценку в виде низкой вероятности для целевого класса:
\begin{align}
J(\Theta) = -\frac{1}{m}\sum_{i=1}^m\sum_{k=1}^Ky_k^{(i)}log(\hat{p}_k^{(i)})
\end{align}
где:
$\Theta$ - матрица параметров.  

Значение $y_k^{(i)}$ равно 1, если целевым классом для i-того образца явлется k, иначе оно равно 0.

Вектор-градиент перекрестной энтропии для класса k
\begin{align}
\nabla_{\theta^{(k)}}J(\Theta)=\frac{1}{m}\sum_{i=1}^m(\hat{p}_k^{(i)}-y_k^{(i)})x^{(i)}
\end{align}

*Пакетный градиентный спуск*  
Градиентный спуск - общий алгоритм оптимизации, способный находить оптимальные решения широкого диапазона задач. Основная идея градиентного спуска - итеративно подстраивать параметры для сведения к минимуму функции издержек.
Пакетный градиентный спуск вычисляет на каждом шаге обучения градиент для всей выборке обучающих данных.
Градиентный спуск измеяет локальный градиент функции ошибки применительно к вектору параметров $\theta$ и двигается в направлении убывающего градиента. Как только градиент становится нулевым - значит достигнут минимум.  
Реализация выглядит следующим образом: инициализируется случайный вектор $\theta$. Затем итеративным образом этот вектор обновляется для снижения функции издержек, до тех пор, пока алгоритм не сойдется в минимуме.  
Размер "шагов" определяется гиперпараметром - скоростью обучения.

Значение параметра:
- слишком маленькое значение делает алгоритм медленным и вычислительно дорогим;
- слишком большое значение может привести к тому, что градиентный спуск проскочи минимум или даже будет расходиться.

Главная проблема в том, что мы не знаем топологию простанства функции ошибок, так что при неудачной инициализации и неправильном шаге обучения, функция может не найти глобальный минимум и застрять на плато или свалиться в локальный минимум.

Хорошо, если функци выпуклая и непрерывная.

Частная функция для производной функции издержек:
\begin{align}
\frac{\partial}{\partial \theta_j}MSE(\theta)=\frac{2}{m}\sum_{i=1}^m(\theta^T\cdotp x^{(i)}-y^{(i)})x_j^{(i)}
\end{align}

Вектор-градиент функции издержек:
\begin{align}
\nabla_{\theta}MSE(\theta) = \begin{pmatrix}
\frac{\partial}{\partial \theta_0}MSE(\theta) \\
\frac{\partial}{\partial \theta_1}MSE(\theta) \\
\frac{\partial}{\partial \theta_2}MSE(\theta) \\
... \\
\frac{\partial}{\partial \theta_n}MSE(\theta) 
\end{pmatrix} = \frac{2}{m}X^T\cdotp (X\cdotp \theta - y)
\end{align}
С вектором-градиентом функции издержек работает алгоритм, который называется пакетным градиентным спуском. Он на каждом шаге потребляет всю выборку обучающих данных.  

Шаг градиентного спуска:
\begin{align}
\theta^{t+1} = \theta^t - \eta\nabla_\theta MSE(\theta)
\end{align}


*Раннее прекращение*  

Раннее прекращение - обучение заканчивается, как только ошибка проверки достигает какого-то заранее опредлеенного минимума. В случае использование стохастичиского или мини-пакетного градиентного спуска хорошим решением может быть прекращение обучения после достижения определенного уровня ошибки и сохранения ее на протяжении определенного числа итераций.

*Что нужно сделать, чтобы реализовать алгоритм:*

In [26]:
# создать случайную выборку
import numpy as np
X = np.random.normal(0, 20, (500, 4))
y = np.random.randint(0, 3, 500)

In [32]:
# инициализируем начальные веса
theta = np.random.randn(4,3)
print(theta)

[[ 1.98360771  0.10249044  0.79090348]
 [-0.48348054  1.32470913  1.33417594]
 [ 0.17749465  0.92874324  1.01648191]
 [ 0.05771686 -0.30772955 -2.26498791]]


In [21]:
# логистическая регрессия
def logit(x:float) -> float:
    return(1 / (1 + np.exp(-x)))

In [36]:
def calc_probability(theta:np.ndarray, x:np.ndarray) -> np.ndarray:
    s = np.dot(theta.T, x)
    p = logit(s)
    return(argmax(p))

In [ ]:
def loss_func(p:np.ndarray, y_real:np.ndarray) -> float:
    cost = (-1/500) *((y_real * np.log(p)) +((1 - y_real) * np.log(1-p)))
    total = cost.sum(axis = 0)
    cost_tot = np.concatenate((cost_tot, tot), axis=0)

In [ ]:
# задать гиперпараметры
n_epochs = 1000
eps = 10e-8
stop_iter_epoch = 20
eta = 0.1
cost_tot = np.empty((0,3))

In [28]:
print(X)

[[-18.33656825   3.70015593   7.36283055   0.53265935]
 [  0.69639636  -3.75033067  32.21554467  15.93393174]
 [  7.49030233  16.59109903  20.84258762 -15.66441528]
 ...
 [-17.06393463  24.3622098   12.13766851 -25.09099653]
 [-11.04252674 -33.19810189  11.0844995  -22.07610823]
 [ -6.70149948  37.66717478 -24.99432476  15.70556806]]
